In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

ruta = "F:/PhD/Datos SIATA/Análisis/Descriptivo/"
archivo = "DF"
datos = pd.read_csv(ruta+"2020-02.csv",sep=",")


In [ ]:
#EXTRAIGO DATOS DE LA BASE DE DATOS GRANDE
datos_nodo = datos.loc[datos.loc[:,"codigoSerial"] == 2]
datos_nodo = datos_nodo.sort_values([' fecha', ' hora'])
datos_nodo.reset_index(inplace=True, drop=True)
#print(datos_nodo[0:10])

plt.figure(figsize=(10, 6), dpi=100)
plt.plot(datos_nodo[" pm25_nova"][0:30], color='red', label='Predicted')

#datos_nodo.to_csv(ruta+'Nodo2-feb.csv', header=True, index=False)
print(datos_nodo[[" fecha", " hora", " temperatura", " pm25_nova"," pm25_df"]][0:30])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from funciones_OD import *

ruta = "F:/PhD/Datos SIATA/Análisis/Descriptivo/"
datos_nodo = pd.read_csv(ruta+"Nodo2-feb.csv",sep=",")

outliers = datos_nodo[["fecha", "hora", "pm25_nova"]]
outliers["fechaHora"] = outliers["fecha"] + " " + outliers["hora"]
#outliers["pm25_nova"] = outliers[" pm25_nova"] 

#outliers.head()

outliers = diferencias_pos (outliers,["pm25_nova"],4)
outliers.drop(["fecha", "hora"], axis = 'columns', inplace=True)
#outliers.set_index("fechaHora", inplace=True)
#plt.figure(figsize=(10, 6), dpi=100)
#plt.plot(outliers.pm25_nova_dif[0:100], color='red', label='Predicted')
#outliers["Outlier"] = False
outliers["Outlier"] = outliers.pm25_nova_dif > 10
outliers.head()
#outliers.to_csv(ruta+'Nodo2-feb-nova-outliers.csv', header=True, index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from funciones_OD import *

ruta = "F:/PhD/Datos SIATA/Análisis/Descriptivo/"
outliers = pd.read_csv(ruta+"Nodo2-feb-nova-outliers.csv",sep=",")

#df_window.loc[df_window["pm25_nova"]>999,"pm25_nova"]=np.nan
#df_window.loc[df_window["pm25_nova"]<0,"pm25_nova"]=np.nan

a = 4290
b = 4400

outliers1 = outliers[a:b][outliers['pm25_outlier'] == True]
plt.figure(figsize=(10, 6), dpi=100)
plt.plot(outliers[a:b].index, outliers[a:b].pm25, color="green",label="pm25")
plt.plot(outliers1.index, outliers1.pm25, 'o', color="red",label="Outliers")
plt.title('HBOS Outlier Detection')
plt.legend()
plt.xlabel('Data')
plt.ylabel('Value')

In [ ]:
# VERIFICO SI UN SENSOR TIENE LOS DATOS COMPLETOS, Y LLENO CON NaN LOS MINUTOS EN LOS QUE NO HA VALOR. ESTO CON LA IDEA DE COMPARAR EN CUÁLES MINUTOS EXISTEN DATOS DE TODOS LOS SENSORES. EL RESULTADO SE ENTREGA EN UN DATASET CON LOS VALORES DE CADA SENSOR EN CADA COLUMNA.
data = pd.concat([datos.iloc[:,0:6], entreno])
days = len(data.fecha.unique().tolist())

data_check = pd.DataFrame()
date = [datetime.datetime(2020,2,1,00,00,00,00) + datetime.timedelta(minutes=minutes) for minutes in range(1440*days)]
data_check["fechaHora"] = pd.to_datetime(date)

for n in data.codigoSerial.unique().tolist():
    nodo = data.loc[data.loc[:,"codigoSerial"]== n][['fecha','hora','pm25']]
    nodo["fechaHora"] = pd.to_datetime(nodo["fecha"] + " " + nodo["hora"])
    nodo = nodo[['fechaHora','pm25']]

    data_check = pd.merge(data_check, nodo, on='fechaHora',how='outer')
    data_check = data_check.rename(columns={'pm25':n})

#data_check.set_index("fechaHora", inplace=True)
data_check

In [ ]:
# EXTRAE VALORES DE UN DATASET OMITIENDO LOS VALORES NaN PARA GRAFICAR UN BOXPLOT SOLO CON LOS DATOS EXISTENTES POR CADA SENSOR, EN UN RANGO DE A HASTA B. AL FINAL, SE CALCULA LA PRUEBA DE TUKEY CON LOS DATOS DE LOS SENSORES PARA CALCULAR LIMITES SUPERIOR E INFERIOR DE LA MEDIA DE LOS DATOS.
# SE TRABAJA UNA FORMA DE CONVERTIR UNA CADENA DE TEXTO EN UN COMANDO YQ UE PYTHON LO EJECUTE (tukey = eval(command)), DONDE COMMAND ES LA CADENA DE TEXTO.

from scipy.stats import tukey_hsd
import matplotlib.pyplot as plt

from statsmodels.stats.multicomp import pairwise_tukeyhsd

a = 0
b = 7


groups = {}
groupsL = ''
for n in data_check.columns[1:].to_list():
    lista = data_check.loc[a:b,n].to_list()
    if np.count_nonzero(np.isnan(lista)) <= (b-a):
        groups[n] = [x for x in lista if np.isnan(x) == False]
        groupsL = groupsL + str([x for x in lista if np.isnan(x) == False]) + ','

plt.boxplot(groups.values())
plt.xticks(range(1, len(groups.keys()) + 1), groups.keys())
plt.ylabel("mean") 
plt.show()

command = 'tukey_hsd('+groupsL+')'

tukey = eval(command)
print(tukey)


'''
groups = ""
groupsL = ""
for n in data_check.columns[1:].to_list():
    lista = data_check.loc[a:b,n].to_list()
    if np.count_nonzero(np.isnan(lista)) <= (b-a):
        globals()[f"group_{n}"] = [x for x in lista if np.isnan(x) == False]
        groups = groups + f"group_{n},"
        groupsL = groupsL + f'"group_{n}",'

print(groups)
print(groupsL)

plt.boxplot(eval(groups))
plt.xticks(range(1, 5), exec(groupsL))
plt.ylabel("mean") 
plt.show()
'''

In [ ]:
#CICLO FOR EN UNA LÍNEA TOMANDO DOS VARIABLES DIFERENTES DE DOS ARREGLOS DE DATOS DIFERENTES.


dos = data_check.loc[a:b,2].to_list()
cincuenta = data_check.loc[a:b,50].to_list()
#print(dos, cincuenta)
diferencia = [e1 - e2 for e1, e2 in zip(dos,cincuenta)]
#print(diferencia)

In [ ]:
# GENERAR UNA LISTA DE VALORES ALEATORIOS A PARTIR DE OTRA LISTA, REPITIENDO LOS VALORES UNA CANTIDAD DE VECES DADA

import numpy as np
from collections import Counter

rep = 5  # Número de veces que se repietirá cada valor
valores = [5, 10, 20]
#valores = [.01, .02, .05, .07, .09, .1] # Lista de valores a repetir y ordenar de forma aleatoria
lista = []  # Lista vacia en la que se alacenaran los valores ordenados aleatoriamente

for num in range(rep*len(valores)):
    val = np.random.choice(valores)
    lista.append(val)
    if lista.count(val) == rep:
        valores.remove(val)

print(lista)
print(valores)


In [1]:
import pandas as pd
from funciones_OD import *

ruta = "F:/PhD/Datos SIATA/Análisis/Descriptivo/"
archivo = "DF"

sensor = "pm25"
redondeo = 4
variables = ['pm25']#,'humedad_relativa', 'temperatura'

entreno = pd.read_csv(ruta+"nodos_entrenamiento_sin_outliers_"+archivo+".csv",sep=",")

porcentaje = 0.6
entreno, datos = df_mix (entreno,porcentaje) 

for sensor in variables:
    datos[sensor] = pd.Series([round(val,redondeo) for val in datos[sensor]])
    entreno[sensor] = pd.Series([round(val,redondeo) for val in entreno[sensor]])

datos_ori, datos, outliers = synthetic_data(datos,0.1,100)

Se incluyeron 6070 Outliers


In [3]:
datos

,codigoSerial,fecha,hora,temperatura,humedad_relativa,pm25,pm25_outlier
0,50,2020-02-18,00:00:00,22.98,80.12,28.5772,N
1,50,2020-02-18,00:01:00,22.97,80.20,27.4039,N
2,50,2020-02-18,00:02:00,22.94,80.29,28.5772,N
3,50,2020-02-18,00:03:00,22.94,80.50,28.5772,N
4,50,2020-02-18,00:05:00,22.93,80.74,28.5772,N
...,...,...,...,...,...,...,...
60691,266,2020-02-08,23:52:00,18.32,81.40,135.4713,35.4713
60692,266,2020-02-08,23:53:00,18.34,81.53,36.5387,N
60693,266,2020-02-08,23:54:00,18.32,81.48,34.9376,N
60694,266,2020-02-08,23:55:00,18.32,81.54,134.9376,34.9376
